# 學習資源推薦系統專案---恆逸網站爬蟲

# 1.爬取數據分析與商業智慧 Data Analysis & Business Intelligence課程

程式碼說明:

1. **讀取並解析文件內容**：
    - 打開並讀取指定文件的內容，將其存儲在`file_content`變量中。

2. **使用正則表達式查找目標節點**：
    - 使用正則表達式在文件內容中查找"數據分析與商業智慧 Data Analysis & Business Intelligence"及其子節點。

3. **解析匹配的子節點並生成課程網址**：
    - 提取並解析匹配的子節點為JSON格式，生成每個課程的URL列表。

4. **定義爬取課程詳細信息的函數**：
    - 從給定的URL爬取課程詳細信息，包括標題、課程時長、課程費用、適合對象和課程大綱。

5. **爬取所有課程並保存到Excel文件**：
    - 遍歷所有URL，爬取課程詳細信息，將所有課程信息保存到Excel文件`courses_details4.xlsx`中。
    

In [ ]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# 使用正則表達式找到 "數據分析與商業智慧 Data Analysis & Business Intelligence" 的節點及其 children 節點
search_pattern = re.compile(r'{"id":"\{J\}23","typeName":"JobCategory","pkid":23,"text":"數據分析與商業智慧 Data Analysis & Business Intelligence","children":(\[.*?\])', re.DOTALL)
match = search_pattern.search(file_content)

if match:
    # 提取匹配的 children 節點
    start_index = match.start(1)
    bracket_count = 1
    end_index = start_index + 1

    # 查找匹配的結束括號以提取完整的 children 節點
    while bracket_count > 0 and end_index < len(file_content):
        if file_content[end_index] == '[':
            bracket_count += 1
        elif file_content[end_index] == ']':
            bracket_count -= 1
        end_index += 1
    
    children_json = file_content[start_index:end_index]
    
    try:
        # 解析為 JSON 格式
        children_courses = json.loads(children_json)
        
        # 生成課程網址
        base_url = "https://www.uuu.com.tw/Course/Show/"
        urls = []
        for course in children_courses:
            course_id = course['id'].replace("{C}", "")
            friendly_url = course['attributes']['FriendlyUrl']
            full_url = f"{base_url}{course_id}/{friendly_url}"
            urls.append(full_url)
    except json.JSONDecodeError as e:
        print(f"JSON 解碼錯誤: {e}")
else:
    print("未找到 '數據分析與商業智慧 Data Analysis & Business Intelligence' 的相關內容。")
    urls = []

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details4.xlsx', index=False)
    print("Data saved to courses_details4.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(urls)


Title of the webpage: 
	
    ChatGPT與Excel整合實戰


Course duration: 7小時
Course cost: NT$ 8,000
Target audience: 希望學習如何使用Excel進行基本的數據處理的初學者對ChatGPT和Excel整合感興趣的個人與專業人士希望提高Excel數據處理技能，並將ChatGPT整合到他們的工作中希望了解如何使用ChatGPT和Excel提高工作效率
DocumentBOX Outline: ChatGPT簡介和基本原理ChatGPT x Excel公式的應用ChatGPT x Excel函數的應用ChatGPT x Excel功能的應用ChatGPT x Excel巨集的應用
Title of the webpage: 
	
    Power BI Desktop數據分析實戰


Course duration: 21小時
Course cost: NT$ 20,000
Target audience: 想要深入了解及學習Power BI Desktop技術，以豐富資料分析報表者數據及報表分析人員需要跨部溝通及討論商業運作者想要學習數據分析及商業思考的使用者熟悉SQL Server報表的建立者已完成以下課程所具備技術能力BITBG：商業數據分析-Excel 2021 Power BI大數據資料清理、拆解分析與視覺化溝通
DocumentBOX Outline: Power BI產品組合介紹Power BI Desktop的下載與安裝Power BI Desktop資料的匯入不同資料格式的匯入及合併彙整資料夾中多個相同格式的資料檔資料的清理、淨化、重塑及轉換資料表間的合併及附加處理查詢步驟的組態與調整查詢的複製、重複及參照資料錯誤的解決與處理Power BI Desktop資料建模與管理組態及優化資料表間的關聯性強化資料模型中的計算邏輯與格式DAX語法說明及各類函數的使用利用DAX公式建立資料表、量值及資料行，以豐富資料模型的處理Power BI Desktop範本檔的建立Power BI Desktop互動式視覺化報表的建立建立豐富的互動式視覺化儀表板多種視覺化效果(卡片、表格、量測計、矩陣、長條圖、組合圖 ......)的運用搭配介紹各項視覺化效果的細部組態視覺化效

結果說明:

1. 會把爬蟲取到的資料儲存成excel檔，並分別使用預設標題。


# 2. 更換search_pattern就可以爬取不同的系列課程資訊

### 爬取資訊安全 Security 系列課程

search_pattern = re.compile(r'{"id":"\{J\}23","typeName":"JobCategory","pkid":23,"text":"數據分析與商業智慧 Data Analysis & Business Intelligence","children":(\[.*?\])', re.DOTALL) 
________________________________________________________________________________________________________________
search_pattern = re.compile(r'{"id":"\{J\}24","typeName":"JobCategory","pkid":24,"text":"資訊安全 Security","children":(\[.*?\])', re.DOTALL)

In [ ]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# 使用正則表達式找到 "資訊安全 Security" 的節點及其 children 節點
search_pattern = re.compile(r'{"id":"\{J\}24","typeName":"JobCategory","pkid":24,"text":"資訊安全 Security","children":(\[.*?\])', re.DOTALL)
match = search_pattern.search(file_content)

if match:
    # 提取匹配的 children 節點
    start_index = match.start(1)
    bracket_count = 1
    end_index = start_index + 1

    # 查找匹配的結束括號以提取完整的 children 節點
    while bracket_count > 0 and end_index < len(file_content):
        if file_content[end_index] == '[':
            bracket_count += 1
        elif file_content[end_index] == ']':
            bracket_count -= 1
        end_index += 1
    
    children_json = file_content[start_index:end_index]
    
    try:
        # 解析為 JSON 格式
        children_courses = json.loads(children_json)
        
        # 生成課程網址
        base_url = "https://www.uuu.com.tw/Course/Show/"
        urls = []
        for course in children_courses:
            course_id = course['id'].replace("{C}", "")
            friendly_url = course['attributes']['FriendlyUrl']
            full_url = f"{base_url}{course_id}/{friendly_url}"
            urls.append(full_url)
    except json.JSONDecodeError as e:
        print(f"JSON 解碼錯誤: {e}")
else:
    print("未找到 '資訊安全 Security' 的相關內容。")
    urls = []

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details5.xlsx', index=False)
    print("Data saved to courses_details5.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(urls)


Title of the webpage: 
	
    CyberOps Associate認證-思科網路安全運營基礎


Course duration: 35小時
Course cost: NT$ 50,000
Target audience: 對網路安全維運監控有高度興趣，欲從事相關工作者從事安全維運中心維運相關工作者想要取得Cisco Certificated CyberOps Associate認證者
DocumentBOX Outline: 定義安全維運中心了解網路基礎結構和網路安全監視工具探索數據類型類別了解基本密碼學概念了解常見的TCP / IP攻擊了解端點安全技術了解以威脅為中心的SOC中的事件分析識別用於應對網路威脅的資源了解事件相關性和規範化識別常見的攻擊媒介識別惡意活動識別可疑行為的模式進行安全事件調查使用Playbook模型組織安全監控了解SOC指標了解SOC工作流程和自動化描述事件回應了解VERIS的用法了解Windows作業系統基礎知識了解Linux作業系統基礎
Title of the webpage: 
	
    CCNP Security認證-建置與設定思科身份識別服務引擎


Course duration: 42小時
Course cost: NT$ 59,000
Target audience: 網路安全工程師Cisco ISE管理人員無線網路安全工程師思科系統整合商和合作夥伴
DocumentBOX Outline: 思科ISE體系結構和部署簡介思科ISE策略實施Web身份驗證和訪客服務思科ISE Profiler思科ISE BYOD思科ISE端點合規性服務使用網路訪問設備
Title of the webpage: 
	
    CISA國際電腦稽核師認證研習班


Course duration: 35小時
Course cost: NT$ 40,000
Target audience: 稽核人員、資訊人員及對資訊系統稽核或控制有興趣者
DocumentBOX Outline: 資訊系統稽核流程 (Information System Auditing Process)資訊科技治理與管理 (Governance and Management of IT)資訊系統的取得、開發與建置 (Information Syst

# 3.再改成一開始只要打好課程代碼和系列課程名稱就可以指定爬取

### 設定目標系列課程的ID和名稱
1. course_id = 29
2. course_name = "雲端技術 Cloud - Google Cloud"

In [ ]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 設定目標系列課程的ID和名稱
course_id = 29
course_name = "雲端技術 Cloud - Google Cloud"

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# 使用正則表達式找到目標系列課程的節點及其 children 節點
search_pattern = re.compile(
    rf'{{"id":"\{{J\}}{course_id}","typeName":"JobCategory","pkid":{course_id},"text":"{course_name}","children":(\[.*?\])',
    re.DOTALL
)
match = search_pattern.search(file_content)

if match:
    # 提取匹配的 children 節點
    start_index = match.start(1)
    bracket_count = 1
    end_index = start_index + 1

    # 查找匹配的結束括號以提取完整的 children 節點
    while bracket_count > 0 and end_index < len(file_content):
        if file_content[end_index] == '[':
            bracket_count += 1
        elif file_content[end_index] == ']':
            bracket_count -= 1
        end_index += 1
    
    children_json = file_content[start_index:end_index]
    
    try:
        # 解析為 JSON 格式
        children_courses = json.loads(children_json)
        
        # 生成課程網址
        base_url = "https://www.uuu.com.tw/Course/Show/"
        urls = []
        for course in children_courses:
            course_id = course['id'].replace("{C}", "")
            friendly_url = course['attributes']['FriendlyUrl']
            full_url = f"{base_url}{course_id}/{friendly_url}"
            urls.append(full_url)
    except json.JSONDecodeError as e:
        print(f"JSON 解碼錯誤: {e}")
else:
    print(f"未找到 '{course_name}' 的相關內容。")
    urls = []

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details_7.xlsx', index=False)
    print("Data saved to courses_details_7.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(urls)


Title of the webpage: 
	
    Google Cloud基礎知識之核心基礎架構


Course duration: 7小時
Course cost: NT$ 14,000
Target audience: 計劃在Google Cloud Platform上部署應用程式和建立應用程式環境的使用者初接觸Google Cloud Platform的開發人員、系統維運專家和解決方案架構師管理或企業決策者等，想評估Google Cloud Platform滿足其各項商業需求的潛力
DocumentBOX Outline: 介紹Google Cloud Platform●  Google Cloud Platform的優勢●  定義Google網路基礎架構的組成，包括：服務據點、資料中心、地區和區域●  了解基礎架構即服務(IaaS)和平台即服務(PaaS)之間的區別GCP的資源與操作●  定義GCP專案的目的●  了解身分識別和存取權管理的目的和使用案例●  列舉操作GCP的方法雲端的VM虛擬機器和網路●  Google Compute Engine的用途和使用案例●  定義基本的GCP網路雲端的儲存服務●  介紹Google Cloud Storage、Google Cloud SQL、Google Cloud Bigtable和Google Cloud Datastore用途和使用案例●  於GCP上挑選合適的儲存產品雲端的容器服務(Containers)●  定義容器的概念並了解容器的用途●  Google Kubernetes Engine和Kubernetes的用途和使用案例雲端的App應用程式●  了解Google App Engine的用途和使用案例●  對比App Engine標準與彈性環境●  了解Google Cloud Endpoints的用途和使用案例：後端架構上開發、部署及管理雲端的開發、部署●  雲端原始碼託管儲存體與Cloud Function●  利用範本建立和管理資源雲端的日誌與監控●  定義SLIs、SLOs、SLAs●  各項整合管理工具：監控、警示與除錯
Title of the webpage: 
	
    Google Cloud的安全性設定


Course duration: 24小時
Course

# 4.爬取所有課程

In [ ]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 設定目標系列課程的ID和名稱
courses = [
    (1, "網路系統工程 System Engineer"),
    (15, "網路通訊 System Communication"),
    (16, "雲端技術 Cloud - Microsoft Azure"),
    (17, "雲端技術 Cloud - Amazon Web Services (AWS)"),
    (18, "虛擬化技術"),
    (19, "程式設計 Programer"),
    (20, "前端工程師系列 Front-End Engineer"),
    (21, "智慧型手機應用程式 Mobile App"),
    (22, "資料庫管理 Database"),
    (23, "數據分析與商業智慧 Data Analysis & Business Intelligence"),
    (24, "資訊安全 Security"),
    (25, "管理類別 Management"),
    (26, "主導稽核員 Leading auditors"),
    (27, "多媒體設計 Multimedia Design"),
    (28, "區塊鏈"),
    (29, "雲端技術 Cloud - Google Cloud")
]

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

all_course_urls = []

for course_id, course_name in courses:
    # 使用正則表達式找到目標系列課程的節點及其 children 節點
    search_pattern = re.compile(
        rf'{{"id":"\{{J\}}{course_id}","typeName":"JobCategory","pkid":{course_id},"text":"{course_name}","children":(\[.*?\])',
        re.DOTALL
    )
    match = search_pattern.search(file_content)

    if match:
        # 提取匹配的 children 節點
        start_index = match.start(1)
        bracket_count = 1
        end_index = start_index + 1

        # 查找匹配的結束括號以提取完整的 children 節點
        while bracket_count > 0 and end_index < len(file_content):
            if file_content[end_index] == '[':
                bracket_count += 1
            elif file_content[end_index] == ']':
                bracket_count -= 1
            end_index += 1
        
        children_json = file_content[start_index:end_index]
        
        try:
            # 解析為 JSON 格式
            children_courses = json.loads(children_json)
            
            # 生成課程網址
            base_url = "https://www.uuu.com.tw/Course/Show/"
            urls = []
            for course in children_courses:
                course_id = course['id'].replace("{C}", "")
                friendly_url = course['attributes']['FriendlyUrl']
                full_url = f"{base_url}{course_id}/{friendly_url}"
                urls.append(full_url)
            
            all_course_urls.extend(urls)
        except json.JSONDecodeError as e:
            print(f"JSON 解碼錯誤: {e}")
    else:
        print(f"未找到 '{course_name}' 的相關內容。")

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details_all.xlsx', index=False)
    print("Data saved to courses_details_all.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(all_course_urls)


未找到 '雲端技術 Cloud - Amazon Web Services (AWS)' 的相關內容。
Title of the webpage: 
	
    CCNA認證-建置與管理Cisco解決方案


Course duration: 70小時
Course cost: NT$ 66,000
Target audience: 1.對通訊產業有高度興趣，欲從事相關工作者2.從事網路維運相關工作者3.想要取得CCNA認證者
DocumentBOX Outline: 1.建立網路與區域網路的基礎概念2.深入瞭解TCP/IP通訊協定（包含新一代IPv6通訊協定介紹）3.認識封包的基本構造4.網路設備的設定與IOS的管理5.建置企業交換器虛擬區域網路(VLAN)環境6.交換器擴展樹協定（STP與RSTP）運作原理與最佳化7.乙太通道網路EtherChannel的運作概念與設定8.路由運作基礎與設定9.廣域網路WAN與虛擬私人網路VPN相關運作技術與設定介紹10.網路設備安全性強化措施，ACL相關技術的實作與設定11.Layer 3 Redundancy的相關技術介紹與HSRP的實作12.淺談智能網路（SDN、SD-Access、SD-WAN、Cisco DNA Center）的發展與變革13.常用網管協定的設定與自動化，例如：SNMP、Syslog、NTP、CDP/LLDP14.網路服務效能（QoS）簡介15.無線區域網路（WLAN）運作原理與基本設定16.認識網路安全威脅與相關的防範因應措施
Title of the webpage: 
	
    CCNP Enterprise認證-思科企業網路建置專家


Course duration: 98小時
Course cost: NT$ 129,000
Target audience: 企業網路架構管理與規劃人員企業網路解決方案規劃與支援人員對大型企業網路運作技術有興趣深入瞭解者
DocumentBOX Outline: ENCOR—思科企業網路核心技術建置與維運ENARSI—建置思科高級企業等級路由與服務
Title of the webpage: 
	
    CCNP Security認證-思科網路安全核心技術建置與維運


Course duration: 56小時
Course cost: NT$ 79,0

## 5. 透過關鍵字查找相關課程

### 使用方式:
### 更換 keyword = "Cloud"，最後會存成 "courses_details_Cloud.xlsx"


In [ ]:
# 爬取所有課程並保存到一個列表中
def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並返回包含課程詳細信息的列表。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    return courses_data

# 使用生成的課程網址爬取課程詳細信息
all_courses_data = scrape_courses(all_course_urls)

# 根據關鍵字過濾課程並保存到Excel文件
def save_courses_to_excel(courses_data, keyword):
    """
    根據關鍵字過濾課程並保存到Excel文件中。
    """
    filtered_courses = [course for course in courses_data if keyword.lower() in course['Title'].lower()]
    df = pd.DataFrame(filtered_courses)
    df.to_excel(f'courses_details_{keyword}.xlsx', index=False)
    print(f"Data saved to courses_details_{keyword}.xlsx")

# 示例關鍵字查找
keyword = "Cloud"
save_courses_to_excel(all_courses_data, keyword)

Title of the webpage: 
	
    CCNA認證-建置與管理Cisco解決方案


Course duration: 70小時
Course cost: NT$ 66,000
Target audience: 1.對通訊產業有高度興趣，欲從事相關工作者2.從事網路維運相關工作者3.想要取得CCNA認證者
DocumentBOX Outline: 1.建立網路與區域網路的基礎概念2.深入瞭解TCP/IP通訊協定（包含新一代IPv6通訊協定介紹）3.認識封包的基本構造4.網路設備的設定與IOS的管理5.建置企業交換器虛擬區域網路(VLAN)環境6.交換器擴展樹協定（STP與RSTP）運作原理與最佳化7.乙太通道網路EtherChannel的運作概念與設定8.路由運作基礎與設定9.廣域網路WAN與虛擬私人網路VPN相關運作技術與設定介紹10.網路設備安全性強化措施，ACL相關技術的實作與設定11.Layer 3 Redundancy的相關技術介紹與HSRP的實作12.淺談智能網路（SDN、SD-Access、SD-WAN、Cisco DNA Center）的發展與變革13.常用網管協定的設定與自動化，例如：SNMP、Syslog、NTP、CDP/LLDP14.網路服務效能（QoS）簡介15.無線區域網路（WLAN）運作原理與基本設定16.認識網路安全威脅與相關的防範因應措施
Title of the webpage: 
	
    CCNP Enterprise認證-思科企業網路建置專家


Course duration: 98小時
Course cost: NT$ 129,000
Target audience: 企業網路架構管理與規劃人員企業網路解決方案規劃與支援人員對大型企業網路運作技術有興趣深入瞭解者
DocumentBOX Outline: ENCOR—思科企業網路核心技術建置與維運ENARSI—建置思科高級企業等級路由與服務
Title of the webpage: 
	
    CCNP Security認證-思科網路安全核心技術建置與維運


Course duration: 56小時
Course cost: NT$ 79,000
Target audience: 負責企業網路安全維護管理者需要企業網路安全相關知識與技術者對於企

In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# 使用正則表達式找到 "數據分析與商業智慧 Data Analysis & Business Intelligence" 的節點及其 children 節點
search_pattern = re.compile(r'{"id":"\{J\}23","typeName":"JobCategory","pkid":23,"text":"數據分析與商業智慧 Data Analysis & Business Intelligence","children":(\[.*?\])', re.DOTALL)
match = search_pattern.search(file_content)

if match:
    # 提取匹配的 children 節點
    start_index = match.start(1)
    bracket_count = 1
    end_index = start_index + 1

    # 查找匹配的結束括號以提取完整的 children 節點
    while bracket_count > 0 and end_index < len(file_content):
        if file_content[end_index] == '[':
            bracket_count += 1
        elif file_content[end_index] == ']':
            bracket_count -= 1
        end_index += 1
    
    children_json = file_content[start_index:end_index]
    
    try:
        # 解析為 JSON 格式
        children_courses = json.loads(children_json)
        
        # 生成課程網址
        base_url = "https://www.uuu.com.tw/Course/Show/"
        urls = []
        for course in children_courses:
            course_id = course['id'].replace("{C}", "")
            friendly_url = course['attributes']['FriendlyUrl']
            full_url = f"{base_url}{course_id}/{friendly_url}"
            urls.append(full_url)
    except json.JSONDecodeError as e:
        print(f"JSON 解碼錯誤: {e}")
else:
    print("未找到 '數據分析與商業智慧 Data Analysis & Business Intelligence' 的相關內容。")
    urls = []

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details4.xlsx', index=False)
    print("Data saved to courses_details4.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(urls)


Title of the webpage: 
	
    ChatGPT與Excel整合實戰


Course duration: 7小時
Course cost: NT$ 8,000
Target audience: 希望學習如何使用Excel進行基本的數據處理的初學者對ChatGPT和Excel整合感興趣的個人與專業人士希望提高Excel數據處理技能，並將ChatGPT整合到他們的工作中希望了解如何使用ChatGPT和Excel提高工作效率
DocumentBOX Outline: ChatGPT簡介和基本原理ChatGPT x Excel公式的應用ChatGPT x Excel函數的應用ChatGPT x Excel功能的應用ChatGPT x Excel巨集的應用
Title of the webpage: 
	
    Power BI Desktop數據分析實戰


Course duration: 21小時
Course cost: NT$ 20,000
Target audience: 想要深入了解及學習Power BI Desktop技術，以豐富資料分析報表者數據及報表分析人員需要跨部溝通及討論商業運作者想要學習數據分析及商業思考的使用者熟悉SQL Server報表的建立者已完成以下課程所具備技術能力BITBG：商業數據分析-Excel 2021 Power BI大數據資料清理、拆解分析與視覺化溝通
DocumentBOX Outline: Power BI產品組合介紹Power BI Desktop的下載與安裝Power BI Desktop資料的匯入不同資料格式的匯入及合併彙整資料夾中多個相同格式的資料檔資料的清理、淨化、重塑及轉換資料表間的合併及附加處理查詢步驟的組態與調整查詢的複製、重複及參照資料錯誤的解決與處理Power BI Desktop資料建模與管理組態及優化資料表間的關聯性強化資料模型中的計算邏輯與格式DAX語法說明及各類函數的使用利用DAX公式建立資料表、量值及資料行，以豐富資料模型的處理Power BI Desktop範本檔的建立Power BI Desktop互動式視覺化報表的建立建立豐富的互動式視覺化儀表板多種視覺化效果(卡片、表格、量測計、矩陣、長條圖、組合圖 ......)的運用搭配介紹各項視覺化效果的細部組態視覺化效

結果說明:

1. 會把爬蟲取到的資料儲存成excel檔，並分別使用預設標題。


# 2. 更換search_pattern就可以爬取不同的系列課程資訊

### 爬取資訊安全 Security 系列課程

search_pattern = re.compile(r'{"id":"\{J\}23","typeName":"JobCategory","pkid":23,"text":"數據分析與商業智慧 Data Analysis & Business Intelligence","children":(\[.*?\])', re.DOTALL) 
________________________________________________________________________________________________________________
search_pattern = re.compile(r'{"id":"\{J\}24","typeName":"JobCategory","pkid":24,"text":"資訊安全 Security","children":(\[.*?\])', re.DOTALL)

In [6]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# 使用正則表達式找到 "資訊安全 Security" 的節點及其 children 節點
search_pattern = re.compile(r'{"id":"\{J\}24","typeName":"JobCategory","pkid":24,"text":"資訊安全 Security","children":(\[.*?\])', re.DOTALL)
match = search_pattern.search(file_content)

if match:
    # 提取匹配的 children 節點
    start_index = match.start(1)
    bracket_count = 1
    end_index = start_index + 1

    # 查找匹配的結束括號以提取完整的 children 節點
    while bracket_count > 0 and end_index < len(file_content):
        if file_content[end_index] == '[':
            bracket_count += 1
        elif file_content[end_index] == ']':
            bracket_count -= 1
        end_index += 1
    
    children_json = file_content[start_index:end_index]
    
    try:
        # 解析為 JSON 格式
        children_courses = json.loads(children_json)
        
        # 生成課程網址
        base_url = "https://www.uuu.com.tw/Course/Show/"
        urls = []
        for course in children_courses:
            course_id = course['id'].replace("{C}", "")
            friendly_url = course['attributes']['FriendlyUrl']
            full_url = f"{base_url}{course_id}/{friendly_url}"
            urls.append(full_url)
    except json.JSONDecodeError as e:
        print(f"JSON 解碼錯誤: {e}")
else:
    print("未找到 '資訊安全 Security' 的相關內容。")
    urls = []

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details5.xlsx', index=False)
    print("Data saved to courses_details5.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(urls)


Title of the webpage: 
	
    CyberOps Associate認證-思科網路安全運營基礎


Course duration: 35小時
Course cost: NT$ 50,000
Target audience: 對網路安全維運監控有高度興趣，欲從事相關工作者從事安全維運中心維運相關工作者想要取得Cisco Certificated CyberOps Associate認證者
DocumentBOX Outline: 定義安全維運中心了解網路基礎結構和網路安全監視工具探索數據類型類別了解基本密碼學概念了解常見的TCP / IP攻擊了解端點安全技術了解以威脅為中心的SOC中的事件分析識別用於應對網路威脅的資源了解事件相關性和規範化識別常見的攻擊媒介識別惡意活動識別可疑行為的模式進行安全事件調查使用Playbook模型組織安全監控了解SOC指標了解SOC工作流程和自動化描述事件回應了解VERIS的用法了解Windows作業系統基礎知識了解Linux作業系統基礎
Title of the webpage: 
	
    CCNP Security認證-建置與設定思科身份識別服務引擎


Course duration: 42小時
Course cost: NT$ 59,000
Target audience: 網路安全工程師Cisco ISE管理人員無線網路安全工程師思科系統整合商和合作夥伴
DocumentBOX Outline: 思科ISE體系結構和部署簡介思科ISE策略實施Web身份驗證和訪客服務思科ISE Profiler思科ISE BYOD思科ISE端點合規性服務使用網路訪問設備
Title of the webpage: 
	
    CISA國際電腦稽核師認證研習班


Course duration: 35小時
Course cost: NT$ 40,000
Target audience: 稽核人員、資訊人員及對資訊系統稽核或控制有興趣者
DocumentBOX Outline: 資訊系統稽核流程 (Information System Auditing Process)資訊科技治理與管理 (Governance and Management of IT)資訊系統的取得、開發與建置 (Information Syst

# 3.再改成一開始只要打好課程代碼和系列課程名稱就可以指定爬取

### 設定目標系列課程的ID和名稱
1. course_id = 29
2. course_name = "雲端技術 Cloud - Google Cloud"

In [8]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 設定目標系列課程的ID和名稱
course_id = 29
course_name = "雲端技術 Cloud - Google Cloud"

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# 使用正則表達式找到目標系列課程的節點及其 children 節點
search_pattern = re.compile(
    rf'{{"id":"\{{J\}}{course_id}","typeName":"JobCategory","pkid":{course_id},"text":"{course_name}","children":(\[.*?\])',
    re.DOTALL
)
match = search_pattern.search(file_content)

if match:
    # 提取匹配的 children 節點
    start_index = match.start(1)
    bracket_count = 1
    end_index = start_index + 1

    # 查找匹配的結束括號以提取完整的 children 節點
    while bracket_count > 0 and end_index < len(file_content):
        if file_content[end_index] == '[':
            bracket_count += 1
        elif file_content[end_index] == ']':
            bracket_count -= 1
        end_index += 1
    
    children_json = file_content[start_index:end_index]
    
    try:
        # 解析為 JSON 格式
        children_courses = json.loads(children_json)
        
        # 生成課程網址
        base_url = "https://www.uuu.com.tw/Course/Show/"
        urls = []
        for course in children_courses:
            course_id = course['id'].replace("{C}", "")
            friendly_url = course['attributes']['FriendlyUrl']
            full_url = f"{base_url}{course_id}/{friendly_url}"
            urls.append(full_url)
    except json.JSONDecodeError as e:
        print(f"JSON 解碼錯誤: {e}")
else:
    print(f"未找到 '{course_name}' 的相關內容。")
    urls = []

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details_7.xlsx', index=False)
    print("Data saved to courses_details_7.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(urls)


Title of the webpage: 
	
    Google Cloud基礎知識之核心基礎架構


Course duration: 7小時
Course cost: NT$ 14,000
Target audience: 計劃在Google Cloud Platform上部署應用程式和建立應用程式環境的使用者初接觸Google Cloud Platform的開發人員、系統維運專家和解決方案架構師管理或企業決策者等，想評估Google Cloud Platform滿足其各項商業需求的潛力
DocumentBOX Outline: 介紹Google Cloud Platform●  Google Cloud Platform的優勢●  定義Google網路基礎架構的組成，包括：服務據點、資料中心、地區和區域●  了解基礎架構即服務(IaaS)和平台即服務(PaaS)之間的區別GCP的資源與操作●  定義GCP專案的目的●  了解身分識別和存取權管理的目的和使用案例●  列舉操作GCP的方法雲端的VM虛擬機器和網路●  Google Compute Engine的用途和使用案例●  定義基本的GCP網路雲端的儲存服務●  介紹Google Cloud Storage、Google Cloud SQL、Google Cloud Bigtable和Google Cloud Datastore用途和使用案例●  於GCP上挑選合適的儲存產品雲端的容器服務(Containers)●  定義容器的概念並了解容器的用途●  Google Kubernetes Engine和Kubernetes的用途和使用案例雲端的App應用程式●  了解Google App Engine的用途和使用案例●  對比App Engine標準與彈性環境●  了解Google Cloud Endpoints的用途和使用案例：後端架構上開發、部署及管理雲端的開發、部署●  雲端原始碼託管儲存體與Cloud Function●  利用範本建立和管理資源雲端的日誌與監控●  定義SLIs、SLOs、SLAs●  各項整合管理工具：監控、警示與除錯
Title of the webpage: 
	
    Google Cloud的安全性設定


Course duration: 24小時
Course

# 4.爬取所有課程

In [11]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 設定目標系列課程的ID和名稱
courses = [
    (1, "網路系統工程 System Engineer"),
    (15, "網路通訊 System Communication"),
    (16, "雲端技術 Cloud - Microsoft Azure"),
    (17, "雲端技術 Cloud - Amazon Web Services (AWS)"),
    (18, "虛擬化技術"),
    (19, "程式設計 Programer"),
    (20, "前端工程師系列 Front-End Engineer"),
    (21, "智慧型手機應用程式 Mobile App"),
    (22, "資料庫管理 Database"),
    (23, "數據分析與商業智慧 Data Analysis & Business Intelligence"),
    (24, "資訊安全 Security"),
    (25, "管理類別 Management"),
    (26, "主導稽核員 Leading auditors"),
    (27, "多媒體設計 Multimedia Design"),
    (28, "區塊鏈"),
    (29, "雲端技術 Cloud - Google Cloud")
]

# 讀取上傳的文件內容並解析
file_path = "C:/Users/WCHuang8/Desktop/CourseTrees_Js"

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

all_course_urls = []

for course_id, course_name in courses:
    # 使用正則表達式找到目標系列課程的節點及其 children 節點
    search_pattern = re.compile(
        rf'{{"id":"\{{J\}}{course_id}","typeName":"JobCategory","pkid":{course_id},"text":"{course_name}","children":(\[.*?\])',
        re.DOTALL
    )
    match = search_pattern.search(file_content)

    if match:
        # 提取匹配的 children 節點
        start_index = match.start(1)
        bracket_count = 1
        end_index = start_index + 1

        # 查找匹配的結束括號以提取完整的 children 節點
        while bracket_count > 0 and end_index < len(file_content):
            if file_content[end_index] == '[':
                bracket_count += 1
            elif file_content[end_index] == ']':
                bracket_count -= 1
            end_index += 1
        
        children_json = file_content[start_index:end_index]
        
        try:
            # 解析為 JSON 格式
            children_courses = json.loads(children_json)
            
            # 生成課程網址
            base_url = "https://www.uuu.com.tw/Course/Show/"
            urls = []
            for course in children_courses:
                course_id = course['id'].replace("{C}", "")
                friendly_url = course['attributes']['FriendlyUrl']
                full_url = f"{base_url}{course_id}/{friendly_url}"
                urls.append(full_url)
            
            all_course_urls.extend(urls)
        except json.JSONDecodeError as e:
            print(f"JSON 解碼錯誤: {e}")
    else:
        print(f"未找到 '{course_name}' 的相關內容。")

# 定義爬蟲函數
def scrape_course_details(url):
    """
    爬取給定URL的課程詳細信息。
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.find('title').get_text()
        print('Title of the webpage:', title)
        
        duration_element = soup.find('li', string=lambda x: x and '時數：' in x)
        if duration_element:
            duration = duration_element.get_text(strip=True).split('：')[1]
            print('Course duration:', duration)
        else:
            print('Course duration not found')
        
        cost_element = soup.find('li', string=lambda x: x and '費用：' in x)
        if cost_element:
            cost = cost_element.get_text(strip=True).split('：')[1]
            print('Course cost:', cost)
        else:
            print('Course cost not found')
        
        target_audience = None
        target_audience_div = soup.find('div', class_='Target')
        if target_audience_div:
            target_audience_element = target_audience_div.find('div', class_='Content')
            if target_audience_element:
                target_audience = target_audience_element.get_text(strip=True)
                print('Target audience:', target_audience)
            else:
                print('Target audience content not found')
        else:
            print('Target audience section not found')
        
        documentbox_outline = None
        documentbox_outline_div = soup.find('div', class_='Outline')
        if documentbox_outline_div:
            documentbox_outline_element = documentbox_outline_div.find('div', class_='Content')
            if documentbox_outline_element:
                documentbox_outline = documentbox_outline_element.get_text(strip=True)
                print('DocumentBOX Outline:', documentbox_outline)
            else:
                print('DocumentBOX Outline content not found')
        else:
            print('DocumentBOX Outline section not found')

        return {
            'Title': title,
            'Course Duration': duration,
            'Course Cost': cost,
            'Target Audience': target_audience,
            'DocumentBOX Outline': documentbox_outline
        }
    else:
        print(f'Failed to retrieve the webpage: {url}')
        return None

def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並保存到Excel文件中。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    
    df = pd.DataFrame(courses_data)
    df.to_excel('courses_details_all.xlsx', index=False)
    print("Data saved to courses_details_all.xlsx")

# 使用生成的課程網址爬取課程詳細信息
scrape_courses(all_course_urls)


未找到 '雲端技術 Cloud - Amazon Web Services (AWS)' 的相關內容。
Title of the webpage: 
	
    CCNA認證-建置與管理Cisco解決方案


Course duration: 70小時
Course cost: NT$ 66,000
Target audience: 1.對通訊產業有高度興趣，欲從事相關工作者2.從事網路維運相關工作者3.想要取得CCNA認證者
DocumentBOX Outline: 1.建立網路與區域網路的基礎概念2.深入瞭解TCP/IP通訊協定（包含新一代IPv6通訊協定介紹）3.認識封包的基本構造4.網路設備的設定與IOS的管理5.建置企業交換器虛擬區域網路(VLAN)環境6.交換器擴展樹協定（STP與RSTP）運作原理與最佳化7.乙太通道網路EtherChannel的運作概念與設定8.路由運作基礎與設定9.廣域網路WAN與虛擬私人網路VPN相關運作技術與設定介紹10.網路設備安全性強化措施，ACL相關技術的實作與設定11.Layer 3 Redundancy的相關技術介紹與HSRP的實作12.淺談智能網路（SDN、SD-Access、SD-WAN、Cisco DNA Center）的發展與變革13.常用網管協定的設定與自動化，例如：SNMP、Syslog、NTP、CDP/LLDP14.網路服務效能（QoS）簡介15.無線區域網路（WLAN）運作原理與基本設定16.認識網路安全威脅與相關的防範因應措施
Title of the webpage: 
	
    CCNP Enterprise認證-思科企業網路建置專家


Course duration: 98小時
Course cost: NT$ 129,000
Target audience: 企業網路架構管理與規劃人員企業網路解決方案規劃與支援人員對大型企業網路運作技術有興趣深入瞭解者
DocumentBOX Outline: ENCOR—思科企業網路核心技術建置與維運ENARSI—建置思科高級企業等級路由與服務
Title of the webpage: 
	
    CCNP Security認證-思科網路安全核心技術建置與維運


Course duration: 56小時
Course cost: NT$ 79,0

## 5. 透過關鍵字查找相關課程

### 使用方式:
### 更換 keyword = "Cloud"，最後會存成 "courses_details_Cloud.xlsx"


In [12]:
# 爬取所有課程並保存到一個列表中
def scrape_courses(urls):
    """
    爬取給定URL列表中的所有課程，並返回包含課程詳細信息的列表。
    """
    courses_data = []
    for url in urls:
        course_details = scrape_course_details(url)
        if course_details:
            courses_data.append(course_details)
    return courses_data

# 使用生成的課程網址爬取課程詳細信息
all_courses_data = scrape_courses(all_course_urls)

# 根據關鍵字過濾課程並保存到Excel文件
def save_courses_to_excel(courses_data, keyword):
    """
    根據關鍵字過濾課程並保存到Excel文件中。
    """
    filtered_courses = [course for course in courses_data if keyword.lower() in course['Title'].lower()]
    df = pd.DataFrame(filtered_courses)
    df.to_excel(f'courses_details_{keyword}.xlsx', index=False)
    print(f"Data saved to courses_details_{keyword}.xlsx")

# 示例關鍵字查找
keyword = "Cloud"
save_courses_to_excel(all_courses_data, keyword)

Title of the webpage: 
	
    CCNA認證-建置與管理Cisco解決方案


Course duration: 70小時
Course cost: NT$ 66,000
Target audience: 1.對通訊產業有高度興趣，欲從事相關工作者2.從事網路維運相關工作者3.想要取得CCNA認證者
DocumentBOX Outline: 1.建立網路與區域網路的基礎概念2.深入瞭解TCP/IP通訊協定（包含新一代IPv6通訊協定介紹）3.認識封包的基本構造4.網路設備的設定與IOS的管理5.建置企業交換器虛擬區域網路(VLAN)環境6.交換器擴展樹協定（STP與RSTP）運作原理與最佳化7.乙太通道網路EtherChannel的運作概念與設定8.路由運作基礎與設定9.廣域網路WAN與虛擬私人網路VPN相關運作技術與設定介紹10.網路設備安全性強化措施，ACL相關技術的實作與設定11.Layer 3 Redundancy的相關技術介紹與HSRP的實作12.淺談智能網路（SDN、SD-Access、SD-WAN、Cisco DNA Center）的發展與變革13.常用網管協定的設定與自動化，例如：SNMP、Syslog、NTP、CDP/LLDP14.網路服務效能（QoS）簡介15.無線區域網路（WLAN）運作原理與基本設定16.認識網路安全威脅與相關的防範因應措施
Title of the webpage: 
	
    CCNP Enterprise認證-思科企業網路建置專家


Course duration: 98小時
Course cost: NT$ 129,000
Target audience: 企業網路架構管理與規劃人員企業網路解決方案規劃與支援人員對大型企業網路運作技術有興趣深入瞭解者
DocumentBOX Outline: ENCOR—思科企業網路核心技術建置與維運ENARSI—建置思科高級企業等級路由與服務
Title of the webpage: 
	
    CCNP Security認證-思科網路安全核心技術建置與維運


Course duration: 56小時
Course cost: NT$ 79,000
Target audience: 負責企業網路安全維護管理者需要企業網路安全相關知識與技術者對於企